In [1]:
from acevedo_clss_and_fcns import * 
from torch_geometric.nn import GNNExplainer
from torch_scatter import scatter

device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'
#device = torch.device(device)
print(f"{device = }")
loader_path = "./results/dataloaders/loader_Concen_plus_Fluxes.pt"


loader = torch.load(loader_path)
a_batch         = next(iter(loader.get_train_loader()))
a_graph         = a_batch[0]
batch_size      = len(a_batch.ptr)-1




/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = 'cuda:0'


In [45]:
gc.collect()
torch.cuda.empty_cache() 
class GIN_classifier_to_explain_v2(torch.nn.Module):
    
    def __init__(
        self, 
        n_classes: int,
        n_nodes : int, 
        num_features : int, 
        out_channels: int = 8,
        dropout : float = 0.05, 
        hidden_dim : int = 8, 
        LeakyReLU_slope : float = 0.01,
        num_layers: int = 2
    ):
        super(GIN_classifier_to_explain_v2, self).__init__() # TODO: why SUPER gato? 
        self.n_nodes = n_nodes
        self.n_classes = n_classes
        self.dropout = dropout
        self.num_features = num_features
        self.out_channels = out_channels
        self.GIN_layers =  GIN(in_channels= num_features, hidden_channels= hidden_dim, num_layers= num_layers, 
                               out_channels= out_channels, dropout=dropout,  jk=None, act='LeakyReLU', act_first = False)              
        self.FC1          = Linear(in_features=out_channels, out_features=1, bias=True)
        self.FC2          = Linear(in_features= self.n_nodes, out_features=self.n_classes, bias=True)
        #self.FC          = Linear(in_features=out_channels, out_features=1, bias=True)           
           
        self.leakyrelu  = LeakyReLU(LeakyReLU_slope)#.to('cuda')
    def forward(self, x, edge_index, batch):
        batch_size = batch.unique().__len__()

        x     = self.GIN_layers(x, edge_index)
        x     = x.reshape(batch_size, self.n_nodes, self.out_channels)
        x     = self.FC1(self.leakyrelu(x))
        x     = x.reshape(batch_size,  self.n_nodes)       
        x     = self.FC2(self.leakyrelu(x))    

        return  x #torch.nn.functional.log_softmax(x, dim=1)
    
model           = GIN_classifier_to_explain_v2(
                                            n_nodes = a_graph.num_nodes, 
                                            num_features = a_graph.num_node_features, 
                                            n_classes = a_graph.num_classes,
                                            hidden_dim=2).to(device, non_blocking=True).to(device)

a_batch.to(device)
model(a_batch.x, a_batch.edge_index, a_batch.batch).shape



torch.Size([64, 2])

In [43]:
gc.collect()
torch.cuda.empty_cache() 
explainer                 = GNNExplainer(model, epochs=2, num_hops = None, lr=0.002).to(device, non_blocking=True)
#node_feat_mask, edge_mask = explainer.explain_graph(a_batch.x, a_batch.edge_index)
node_feat_mask, edge_mask = explainer.explain_graph(a_batch[0].x, a_batch[0].edge_index)

#node_feat_mask.shape, edge_mask.shape

Explain graph: 100%|██████████| 2/2 [00:00<00:00, 500.60it/s]


In [34]:
node_feat_mask, edge_mask = explainer.explain_graph(a_batch[0].x, a_batch[0].edge_index)
node_feat_mask.shape, edge_mask.shape

Explain graph: 100%|██████████| 2/2 [00:00<00:00, 299.57it/s]


(torch.Size([]), torch.Size([53122]))

In [9]:
edge_mask.cpu().numpy().min(), edge_mask.cpu().numpy().max()

(0.48584098, 0.5127498)

In [27]:
class GIN_classifier_to_explain(torch.nn.Module):
    
    def __init__(
        self, 
        batch_size,
        #target_node_idx: int,
        n_nodes : int, 
        num_features : int, 
        out_channels: int = 8,
        dropout : float = 0.05, 
        hidden_dim : int = 8, 
        LeakyReLU_slope : float = 0.01,

        num_layers: int = 4
    ):
        super(GIN_classifier_to_explain, self).__init__() # TODO: why SUPER gato? 
        self.n_nodes = n_nodes
        self.dropout = dropout
        self.num_features = num_features
        #self.target_node_idx = target_node_idx
        self.out_channels = out_channels
        self.batch_size   = batch_size
        
        self.GIN_layers =  GIN(in_channels= num_features, hidden_channels= hidden_dim, num_layers= num_layers, 
                               out_channels= out_channels, dropout=dropout,  jk=None, act='LeakyReLU', act_first = False)      
        
        self.FC1        = Linear(in_features=out_channels, out_features=1, bias=True)
        self.FC2        = Linear(in_features=self.n_nodes, out_features=2, bias=True)      
        self.leakyrelu  = LeakyReLU(LeakyReLU_slope)#.to('cuda')
    def forward(self, x, edge_index, batch):


        x     = self.GIN_layers(x, edge_index)
        x     = x.reshape(self.batch_size, self.n_nodes, self.out_channels)
        x     = self.FC1(self.leakyrelu(x))
        x     = x.reshape(self.batch_size,  self.n_nodes)              
        #x     = self.FC2(self.leakyrelu(x))
        #x     = x.reshape(self.batch_size, 2)
        return   x #torch.nn.functional.log_softmax(x, dim=1).squeeze()
    
model           = GIN_classifier_to_explain(batch_size = 64,
                                            n_nodes = a_graph.num_nodes, 
                                            num_features = a_graph.num_node_features, 
                                            #n_classes = a_graph.num_classes,
                                            hidden_dim=2).to(device, non_blocking=True).to(device)
a_batch.to(device)
model(a_batch.x, a_batch.edge_index, a_batch.batch).shape
explainer                 = GNNExplainer(model, epochs=2, num_hops = None, lr=0.002).to(device, non_blocking=True)
node_feat_mask, edge_mask = explainer.explain_graph(a_batch.x, a_batch.edge_index)







Explain graph: 100%|██████████| 2/2 [00:00<00:00, 49.76it/s]


In [19]:
node_feat_mask, edge_mask = explainer.explain_graph(a_batch[0].x, a_batch[0].edge_index)


RuntimeError: shape '[64, 12228, 8]' is invalid for input of size 97824